<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/fluxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up environment

In [ ]:
!pip install cfgrib
!pip install seawater

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import cfgrib
import seawater

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#User parameters

In [ ]:
name = "Milton"
tcNum = "14"
trackType = ""

initTime = dt(year=2024, month=10, day=7, hour=12) #Time when Milton began its most rapid intensification

fHourStep = 3       #Normally 3 for HAFS-A
forecastLength = 60 #Normally 126 for HAFS-A.

figureSuffix = "_RI"
subfolder = "RI/"
dataPath = "/content/drive/MyDrive/savedData/"
figurePath = "/content/drive/MyDrive/figures/"

stormCentered = True
potentialTemp = True #Use atmospheric potential temperature instead of in-situ temperature

atmTop = 850
oceFloor = 100

avgTypes = ["point", "centered", "NE", "SE", "SW", "NW"]
areaRange = 0.5 #in degrees

#Retrieve HAFS-A Data

Find times needed

In [ ]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d%H"

fcastTimes = [] #Key: initiation, item: valid time list
fhour = 0
validTime = initTime

while fhour <= forecastLength:
    fcastTimes.append(validTime)
    validTime += pd.Timedelta(hours=fHourStep)
    fhour += fHourStep

Find storm location in HAFS-A from ATCF files.

In [ ]:
btPath = "/content/drive/MyDrive/savedData/Milton/hurdat2_Milton.csv"
bt = pd.read_csv(btPath)

In [ ]:
cols = ["BASIN", "CY", "YYYYMMDDHH", "TECHNUM/MIN", "TECH", "TAU", "LatN/S", "LonE/W",
    "VMAX", "MSLP", "TY", "RAD", "WINDCODE", "RAD1", "RAD2", "RAD3", "RAD4",
    "POUTER", "ROUTER", "RMW", "GUSTS", "EYE", "SUBREGION", "MAXSEAS", "INITIALS",
    "DIR", "SPEED", "STORMNAME", "DEPTH", "SEAS", "SEASCODE", "SEAS1", "SEAS2",
    "SEAS3", "SEAS4", "USERDEFINED1", "Thermo1", "Thermo2", "Thermo3", "Thermo4",
    "Thermo5", "Thermo6", "Thermo7", "USERDEFINED2", "DT", "SHR82", "SHR81_1",
    "SHR82_2",  "USERDEFINED3", "SST", "USERDEFINED4", "ARMW1", "ARMW2"]

initStr, initHour = initTime.strftime("%Y%m%d_%H").split("_")

atcfURL = f"https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/{initStr}/{initHour}/{tcNum}l.{initStr}{initHour}.hfsa.trak.atcfunix"
atcfFile = "atcf_" + initStr + "_" + initHour + ".csv"

!wget -O {atcfFile} {atcfURL}
atcf = pd.read_csv(atcfFile, names=cols)

In [ ]:
tcLocs = {}
vmax = {}
mslp = {}
vmaxErr = {}
mslpErr = {}

for valid in fcastTimes:
    fHour = int((valid-initTime).total_seconds() / 3600)

    pointLat = int(atcf[atcf.TAU==fHour]["LatN/S"].iloc[0].replace("N", ""))/10
    pointLon = int(atcf[atcf.TAU==fHour]["LonE/W"].iloc[0].replace("W", ""))/-10 #Assume western hemisphere
    tcLocs[valid] = (pointLat, pointLon)
    vmax[valid] = atcf[atcf.TAU==fHour]["VMAX"].iloc[0]
    mslp[valid] = atcf[atcf.TAU==fHour]["MSLP"].iloc[0]
    if valid.hour%6 == 0:
      vmaxErr[valid] = atcf[atcf.TAU==fHour]["VMAX"].iloc[0] - bt[bt.time==str(valid)].vmax.iloc[0]
      mslpErr[valid] = atcf[atcf.TAU==fHour]["MSLP"].iloc[0] - bt[bt.time==str(valid)].mslp.iloc[0]

Get data from HAFS-A output.

In [ ]:
atm = {}
for avgType in avgTypes:

  atmAvg = {}
  for valid in fcastTimes:

    initStr = initTime.strftime("%Y%m%d%H")

    atmAvg[valid] = {}

    fhour = str(int((valid-initTime).total_seconds() / 3600))
    while len(fhour) < 3: fhour = "0" + fhour

    atmFile = "hafsa_" + initStr + "_f" + fhour + ".nc"
    atmPath = dataPath + "hafsaOutput/" + subfolder + atmFile
    atmData = xr.open_dataset(atmPath, decode_timedelta=False)

    point = tcLocs[valid]
    levels = atmData.isobaricInhPa.data[atmData.isobaricInhPa.data>atmTop]

    area = {"NE":[point[0], point[1], point[0]+areaRange, point[1]+areaRange], "SE":[point[0]-areaRange, point[1], point[0], point[1]+areaRange],
            "SW":[point[0]-areaRange, point[1]-areaRange, point[0], point[1],], "NW":[point[0], point[1]-areaRange, point[0]+areaRange, point[1]],
            "centered":[point[0]-(0.5*areaRange), point[1]-(0.5*areaRange), point[0]+(0.5*areaRange), point[1]+(0.5*areaRange)], "point":None}[avgType]

    for level in levels:
      #Longitude in atm files are in degrees east, but are -180 - 180 in oce files. point has them from -180 - 180
      atmAvg[valid][level] = {}

      if avgType == "point": validPoint = atmData.sel(latitude=point[0], longitude=point[1] + 360, isobaricInhPa=level, method="nearest")
      else: validPoint = atmData.sel(latitude=slice(area[0], area[2]), longitude=slice(area[1]+360, area[3]+360), isobaricInhPa=level)

      atmAvg[valid][level]["T"] = np.mean(validPoint.t.data)
      atmAvg[valid][level]["q"] = np.mean(validPoint.q.data) * 1000 #Convert from kg/kg to g/kg
      atmAvg[valid][level]["u"] = np.mean(validPoint.u.data)
      atmAvg[valid][level]["v"] = np.mean(validPoint.v.data)
      atmAvg[valid][level]["gh"] = np.mean(validPoint.gh.data)
      atmAvg[valid][level]["sst"] = np.mean(validPoint.sst.data) #Make opposite so that it represents flux into the ocean
      atmAvg[valid][level]["shf"] = np.mean(validPoint.ishf.data) * -1
      atmAvg[valid][level]["lhf"] = np.mean(validPoint.slhtf.data) * -1
      atmAvg[valid][level]["tau"] = (np.mean(validPoint.utaua.data)**2 + np.mean(validPoint.vtaua.data)**2)**0.5 * -1

  atm[avgType] = atmAvg

In [ ]:
oce = {}

for avgType in avgTypes:
  oceAvg = {}

  for valid in fcastTimes:

    fhour = str(int((valid-initTime).total_seconds() / 3600))
    while len(fhour) < 3: fhour = "0" + fhour
    oceFile = "mom6_" + initStr + "_f" + fhour + ".nc"
    ocePath = dataPath + "mom6Output/" + subfolder + oceFile

    if oceFile == 'mom6_2024100800_f000.nc': #This file is missing
      for layer in oceData.z_l.data:
        oceAvg[initTime][valid][layer] = {"T":np.nan, "s":np.nan, "u":np.nan, "v":np.nan}
      continue

    oceData = xr.open_dataset(ocePath, decode_times=False)

    point = tcLocs[valid]
    levels = oceData.z_l.data[oceData.z_l.data<=oceFloor] #Retrieves most shallow layer; should be 1 m

    oceAvg[valid] = {}

    for level in levels:

      if avgType == "point":
        validPoint = oceData.sel(yh=point[0], yq=point[0], xq=point[1], xh=point[1], z_l=level, method="nearest")
        sfcPoint = oceData.sel(yh=point[0], yq=point[0], xq=point[1], xh=point[1], method="nearest")
      else:
        validPoint = oceData.sel(yq=slice(area[0], area[2]), yh=slice(area[0], area[2]), xq=slice(area[1], area[3]), xh=slice(area[1], area[3]), z_l=level)
        sfcPoint = oceData.sel(yq=slice(area[0], area[2]), yh=slice(area[0], area[2]), xq=slice(area[1], area[3]), xh=slice(area[1], area[3]))

      oceAvg[valid][level] = {}
      oceAvg[valid][level]["T"] = np.mean(validPoint.temp.data) + 273.15 #This is potential temperature. Converted from C to K.
      oceAvg[valid][level]["s"] = np.mean(validPoint.so.data)
      oceAvg[valid][level]["u"] = np.mean(validPoint.uo.data)
      oceAvg[valid][level]["v"] = np.mean(validPoint.vo.data)

      oceAvg[valid][level]["sst"] = np.mean(sfcPoint.SST.data) + 273.15
      oceAvg[valid][level]["ssh"] = np.mean(sfcPoint.SSH.data)
      oceAvg[valid][level]["shf"] = np.mean(sfcPoint.sensible.data)
      oceAvg[valid][level]["lhf"] = np.mean(sfcPoint.latent.data)

    oce[avgType] = oceAvg

#Figures

Goal: Correlate atmospheric sensible and latent heat flues and momentum fluxes with intensity forecasts and error across runs.  
Particularly, look at the wind-caused and current-caused momentum flux.

Fluxes and temperature difference between 1000 hPa and 1 m/SST

In [ ]:
avgType1 = "point"
avgType2 = "centered"

In [ ]:
fluxFig, axes = plt.subplots(4, 1, sharex=True, figsize=(8,8))

labels = ["Sensible Heat Flux", "Latent Heat Flux", "Latent + Sensible Heat Flux", "Wind Momentum Flux"]
ylabels = ["Heat Flux (W/m$^2$)", "Heat Flux (W/m" + "$^2$" + ")", "Heat Flux (W/m" + "$^2$" + ")", "Momentum Flux (N/m" + "$^2$" + ")"]
colors = ["darkorange", "green", "magenta", "teal"]
ylims = [(-350, 50), (-1300, 0), (-1600, 0), (-6, 0)]

tDiff = [oce[avgType1][valid][1.0]["T"] - atm[avgType1][valid][1000.0]["T"] for valid in atm[avgType1].keys()]
sstDiff = [atm[avgType1][valid][1000.0]["sst"] - atm[avgType1][valid][1000.0]["T"] for valid in atm[avgType1].keys()]

shf = [atm[avgType1][valid][1000.0]["shf"] for valid in atm[avgType1].keys()]
lhf = [atm[avgType1][valid][1000.0]["lhf"] for valid in atm[avgType1].keys()]
flux = [atm[avgType1][valid][1000.0]["lhf"]+atm[avgType1][valid][1000.0]["shf"] for valid in atm[avgType1].keys()]
mf = [atm[avgType1][time][1000.0]["tau"] for time in atm[avgType1].keys()]

if avgType2:
  tDiff2 = [oce[avgType2][valid][1.0]["T"] - atm[avgType2][valid][1000.0]["T"] for valid in atm[avgType2].keys()]
  sstDiff2 = [atm[avgType2][valid][1000.0]["sst"] - atm[avgType2][valid][1000.0]["T"] for valid in atm[avgType2].keys()]

  shf2 = [atm[avgType2][valid][1000.0]["shf"] for valid in atm[avgType2].keys()]
  lhf2 = [atm[avgType2][valid][1000.0]["lhf"] for valid in atm[avgType2].keys()]
  flux2 = [atm[avgType2][valid][1000.0]["lhf"]+atm[avgType2][valid][1000.0]["shf"] for valid in atm[avgType2].keys()]
  mf2 = [atm[avgType2][time][1000.0]["tau"] for time in atm[avgType2].keys()]

for ax, flux, flux2, label, ylabel, color, ylim in zip(axes, [shf, lhf, flux, mf], [shf2, lhf2, flux2, mf2], labels, ylabels, colors, ylims):

  tAx = ax.twinx()

  if ax == axes[0]:
    tAx.plot(atm[avgType1].keys(), tDiff, color="blue", linestyle="--", label="T$_o$ - T$_a$")
    tAx.plot(atm[avgType1].keys(), sstDiff, color="red", linestyle="--", label="SST - T$_a$")
  else:
    tAx.plot(atm[avgType1].keys(), tDiff, color="blue", linestyle="--")
    tAx.plot(atm[avgType1].keys(), sstDiff, color="red", linestyle="--")
  ax.plot(atm[avgType1].keys(), flux, label=label, color=color)

  if avgType2:
    tAx.plot(atm[avgType2].keys(), tDiff2, color="blue", linestyle="--", linewidth=2.5, alpha=0.4)
    tAx.plot(atm[avgType2].keys(), sstDiff2, color="red", linestyle="--", linewidth=2.5, alpha=0.4)
    ax.plot(atm[avgType2].keys(), flux2, color=color, linewidth=2, alpha=0.5)

  ax.set_ylim(ylim[0], ylim[1])
  ax.grid(alpha=0.5)
  tAx.set_ylabel("T Difference (K)", color="red")
  ax.set_ylabel(ylabel, color=color)
  ax.set_title("r$_{To}$ = " + str(round(np.corrcoef(tDiff, flux)[0,1], 2)) + ",  r$_{SST}$ = " + str(round(np.corrcoef(sstDiff, flux)[0,1], 2)))
  ax.tick_params(axis='x', rotation=45)

fluxFig.legend(ncols=2)

Profiles and SST

In [ ]:
times = [key for key in atm.keys() if key.hour%6 == 0]
xwidth = (0.60/len(times))
xcorners = np.arange(0.1, 0.9, 0.8/len(times))

In [ ]:
profFig = plt.figure(figsize=(14,4))
profAxes = [profFig.add_axes([xcorner,0.1,xwidth,0.8]) for xcorner in xcorners]

for ax, valid in zip(profAxes, times):
  if valid.hour%6 != 0 : continue
  t = [atm[avgType1][valid][level]["T"] for level in atm[valid].keys()]
  p = atm[avgType2][valid].keys()
  ax.plot(t, p, color="blue", label="Atm. Temp.")
  ax.scatter(atm[valid][1000.0]["sst"], 1000, color="red", s=50, label="SST")

  ax.set_xlim(290, 310)
  ax.invert_yaxis()
  grid = ax.grid(alpha=0.5)
  ax.set_title(valid.strftime("%m-%d %HUTC"))

  if valid == fcastTimes[0]: ax.legend(ncols=2, loc=(0, -0.2))
  else: ax.set_yticklabels([])
  ax.tick_params(axis='x', rotation=45)

profFig.supxlabel("Temperature (K)")
profFig.supylabel("Pressure (hPa)")

Look at evolution of temperature and flux

In [ ]:
fluxFig = plt.figure(figsize=(8,5))

t = list(atm[avgType1].keys())
atmT = [atm[avgType1][valid][1000.0]["T"] - atm[avgType1][past][1000.0]["T"] for (past, valid) in zip(t[:-1], t[1:])]
sst = [atm[avgType1][valid][1000.0]["sst"] - atm[avgType1][past][1000.0]["sst"] for (past, valid) in zip(t[:-1], t[1:])]
oceT = [oce[avgType1][valid][1.0]["T"] - oce[avgType1][past][1.0]["T"] for (past, valid) in zip(t[:-1], t[1:])]
shf = [atm[avgType1][valid][1000.0]["shf"] for valid in t[1:]]
lhf = [atm[avgType1][valid][1000.0]["lhf"] for valid in t[1:]]
flux = [atm[avgType1][valid][1000.0]["lhf"]+atm[avgType1][valid][1000.0]["shf"] for valid in t[1:]]

tAxshf = fluxFig.add_axes([0.1,0.65,0.8,0.25])
shfAx = tAxshf.twinx()
tAxshf.plot(t[1:], atmT, label="$\Delta$T(1000 hPa)", color="red")
tAxshf.plot(t[1:], oceT, label="$\Delta$T(1 m)", color="blue")
tAxshf.plot(t[1:], sst, label="$\Delta$SST", color="purple")
shfAx.plot(t[1:], shf, label="Sensible Heat Flux", color="darkorange")

tAxshf.grid(alpha=0.5)
tAxshf.set_xticklabels([])
#tAxshf.set_title(f"Correlation = {str(round(np.corrcoef(tDiff, shf)[0,1], 2))}")

tAxlhf = fluxFig.add_axes([0.1,0.35,0.8,0.25])
lhfAx = tAxlhf.twinx()
tAxlhf.plot(t[1:], atmT, color="red")
tAxlhf.plot(t[1:], oceT, color="blue")
tAxlhf.plot(t[1:], sst, color="purple")
lhfAx.plot(t[1:], lhf, label="Latent Heat Flux", color="green")

tAxlhf.grid(alpha=0.5)
tAxlhf.set_xticklabels([])
#tAxlhf.set_title(f"Correlation = {str(round(np.corrcoef(tDiff, lhf)[0,1], 2))}")

tAxFlux = fluxFig.add_axes([0.1,0.05,0.8,0.25])
fluxAx = tAxFlux.twinx()

tAxFlux.plot(t[1:], atmT, color="red")
tAxFlux.plot(t[1:], oceT, color="blue")
tAxFlux.plot(t[1:], sst, color="purple")
fluxAx.plot(t[1:], flux, label="Latent + Sensible Heat Flux", color="magenta")

tAxFlux.grid(alpha=0.5)
#tAxFlux.set_title(f"Correlation = {str(round(np.corrcoef(tDiff, flux)[0,1], 2))}")

fluxFig.legend()
fluxFig.supylabel("Atmosphere - Ocean $\Delta$T (K)", ha="left")
fluxFig.supylabel("Heat Flux (W/m" + "$^2$" + ")", ha="right")

In [ ]:
np.corrcoef([atmT, oceT, sst, flux])

Fluxes with intensity

TODO: Add MSLP

In [ ]:
# Create a figure with a subplot for each run time
fig, axes = plt.subplots(4, 1, figsize=(8, 8), sharex=True)

shf = [atm[avgType1][time][1000.0]["shf"] for time in atm[avgType1].keys()]
lhf = [atm[avgType1][time][1000.0]["lhf"] for time in atm[avgType1].keys()]
flux = [atm[avgType1][time][1000.0]["lhf"]+atm[avgType1][time][1000.0]["shf"] for time in atm[avgType1].keys()]
mf = [atm[avgType1][time][1000.0]["tau"] for time in atm[avgType1].keys()]
intensity = [vmax[time] for time in atm[avgType1].keys()] # Get intensity values

labels = ["Sensible Heat Flux", "Latent Heat Flux", "Latent + Sensible Heat Flux", "Wind Momentum Flux"]
ylabels = ["Heat Flux (W/m$^2$)", "Heat Flux (W/m" + "$^2$" + ")", "Heat Flux (W/m" + "$^2$" + ")", "Momentum Flux (N/m" + "$^2$" + ")"]
colors = ["darkorange", "green", "magenta", "teal"]
ylims = [(-350, 0), (-1300, -400), (-1600, -400), (-3, 0)]

for ax, flux, label, ylabel, color, ylim in zip(axes, [shf, lhf, flux, mf], labels, ylabels, colors, ylims):

  intAx = ax.twinx()
  ax.plot(atm[avgType1].keys(), flux, label=label, color=color)
  intAx.plot(atm[avgType1].keys(), intensity, color="black")

  ax.set_ylim(ylim[0], ylim[1])
  intAx.set_ylabel("Intensity (kt)")
  ax.set_ylabel(ylabel, color=color)
  ax.grid(alpha=0.5)
  ax.set_title(f"r = {str(round(np.corrcoef(intensity, flux)[0,1], 2))}")
  ax.tick_params(axis='x', rotation=45)

fig.legend()#loc=[0.01, 0.94], ncols=4)

Fluxes with $\Delta$Intensity (I don't think the corelations here matter)

In [ ]:
# Create a figure with a subplot for each run time
fig, axes = plt.subplots(4, 1, figsize=(8, 8), sharex=True)

validTimes = list(atm[avgType1].keys())[1:]
pastTimes = list(atm[avgType1].keys())[:-1]

shf = [atm[avgType1][time][1000.0]["shf"] for time in validTimes]
lhf = [atm[avgType1][time][1000.0]["lhf"] for time in validTimes]
flux = [atm[avgType1][time][1000.0]["lhf"]+atm[avgType1][time][1000.0]["shf"] for time in validTimes]
mf = [atm[avgType1][time][1000.0]["tau"] for time in validTimes]
dIntensity = [vmax[validTime]-vmax[pastTime] for validTime, pastTime in zip(validTimes, pastTimes)] # Get intensity values

labels = ["Sensible Heat Flux", "Latent Heat Flux", "Latent + Sensible Heat Flux", "Wind Momentum Flux"]
colors = ["darkorange", "green", "magenta", "teal"]
ylims = [(-350, 0), (-1300, -400), (-1600, -400), (-3, 0)]

for ax, flux, label, color, ylim in zip(axes, [shf, lhf, flux, mf], labels, colors, ylims):

  intAx = ax.twinx()
  ax.plot(validTimes, flux, label=label, color=color)
  intAx.plot(validTimes, dIntensity, color="black")

  ax.set_ylim(ylim[0], ylim[1])
  intAx.set_ylabel("$\Delta$Intensity (kt)")
  ax.set_ylabel("Heat Flux (W/m" + "$^2$" + ")", color=color)
  ax.grid(alpha=0.5)
  ax.set_title(f"r = {str(round(np.corrcoef(dIntensity, flux)[0,1], 2))}")

axes[-1].set_xlabel("Time")
axes[-1].set_ylabel("Mom. Flux (N/m" + "$^2$" + ")", color=color)
axes[-1].tick_params(axis='x', rotation=45)

fig.legend(loc=[0.05, 0.97], ncols=4)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Fluxes and forecast error

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(8, 8), sharex=True)

shf = [atm[avgType1][time][1000.0]["shf"] for time in atm[avgType1].keys()]
lhf = [atm[avgType1][time][1000.0]["lhf"] for time in atm[avgType1].keys()]
flux = [atm[avgType1][time][1000.0]["lhf"]+atm[avgType1][time][1000.0]["shf"] for time in atm[avgType1].keys()]
mf = [atm[avgType1][time][1000.0]["tau"] for time in atm[avgType1].keys()]

sixHourly = [time for time in atm[avgType1].keys() if time.hour%6==0]
err = [vmaxErr[time] for time in sixHourly]

labels = ["Sensible Heat Flux", "Latent Heat Flux", "Latent + Sensible Heat Flux", "Wind Momentum Flux"]
ylabels = ["Heat Flux (W/m$^2$)", "Heat Flux (W/m" + "$^2$" + ")", "Heat Flux (W/m" + "$^2$" + ")", "Momentum Flux (N/m" + "$^2$" + ")"]
colors = ["darkorange", "green", "magenta", "teal"]
ylims = [(0, -350), (-400, -1300), (-400, -1600), (0, -3)]

for ax, flux, label, ylabel, color, ylim in zip(axes, [shf, lhf, flux, mf], labels, ylabels, colors, ylims):

  intAx = ax.twinx()
  ax.plot(atm[avgType1].keys(), flux, label=label, color=color)
  intAx.plot(sixHourly, err, color="black")

  ax.set_ylim(ylim[0], ylim[1])
  intAx.set_ylabel("Vmax Error (kt)")
  ax.set_ylabel(ylabel, color=color)
  ax.grid(alpha=0.5)
  ax.set_title(f"r = {str(round(np.corrcoef(err, [flux[i] for i in range(len(atm[avgType1].keys())) if list(atm[avgType1].keys())[i].hour%6==0])[0,1], 2))}")
  ax.tick_params(axis='x', rotation=45)

fig.legend(loc=[0.01, 0.95], ncols=4)